In [1]:
import streamlit as st
import pandas as pd
import geopandas as gpd
import plotly.express as px
from datetime import datetime
import json
import os

In [2]:
# get current working directory
getcwd = os.getcwd()
getcwd

'/Users/dhartipatelseagraves/Documents/GitHub/ProjectToto/Streamlit'

In [4]:
# Set up the Streamlit app
# st.title('Iowa Tornado Risk Map')
# st.write('This app visualizes the tornado risk for each county in Iowa.')

# # Load tornado risk data
# @st.cache_data
def load_tornado_risk_data():
    data = pd.read_csv('tornado_risk.csv')
    data['DateTime'] = pd.to_datetime(data['DateTime'])  # Ensure proper datetime format
    # Trim whitespace from all string columns and convert County Name to uppercase
    data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    data['County Name'] = data['County Name'].str.upper().str.strip()
    data['State'] = data['State'].str.upper().str.strip()
    data = data.rename(columns={'County Name': 'CountyName'})  # Rename column for merging
    return data

tornado_data = load_tornado_risk_data()
print(tornado_data)
# st.write('Tornado Data', tornado_data.head())

/var/folders/z9/y1zmbc2500578321lpfyjq1r0000gn/T/ipykernel_74170/4183535968.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


             Date      Time            DateTime  TornadoRisk CountyName State  \
0      2023-02-26  00:08:13 2023-02-26 00:08:13      0.37454       LYON    IA   
1      2023-02-26  01:44:14 2023-02-26 01:44:14      0.95071       LYON    IA   
2      2023-02-26  02:45:15 2023-02-26 02:45:15      0.73199       LYON    IA   
3      2023-02-26  03:47:50 2023-02-26 03:47:50      0.59866       LYON    IA   
4      2023-02-26  04:44:43 2023-02-26 04:44:43      0.15602       LYON    IA   
...           ...       ...                 ...          ...        ...   ...   
83155  2023-04-01  19:09:22 2023-04-01 19:09:22      0.96818        LEE    IA   
83156  2023-04-01  20:17:20 2023-04-01 20:17:20      0.91216        LEE    IA   
83157  2023-04-01  21:53:46 2023-04-01 21:53:46      0.46365        LEE    IA   
83158  2023-04-01  22:53:12 2023-04-01 22:53:12      0.34490        LEE    IA   
83159  2023-04-01  23:01:01 2023-04-01 23:01:01      0.61264        LEE    IA   

      CountyDisplayName  
0

In [5]:
# Load GeoJSON file
# @st.cache_data
def load_geojson():
    gdf = gpd.read_file('Iowa_County_Boundaries.geojson')
    gdf['CountyName'] = gdf['CountyName'].str.upper().str.strip()
    gdf['StateAbbr'] = gdf['StateAbbr'].str.upper().str.strip()
    gdf = gdf.rename(columns={'StateAbbr': 'State'})  # Rename column for merging
    return gdf

geojson_data = load_geojson()
print(geojson_data)
# st.write('GeoJSON Data', geojson_data.head())

    OBJECTID  PERIMETER  DOMCountyID   FIPS  Shape_Length  Shape_Area  \
0          1   193975.5           60  19119      2.212753    0.169102   
1          2   130929.6           72  19143      1.434543    0.114867   
2          3   131290.5           30  19059      1.438023    0.116176   
3          4   130956.8           32  19063      1.434046    0.115803   
4          5   172890.3            3  19005      1.847862    0.189376   
..       ...        ...          ...    ...           ...         ...   
94        95   144995.3           26  19051      1.526090    0.139393   
95        96   148600.8           27  19053      1.558094    0.147367   
96        97   147785.1           93  19185      1.551256    0.145582   
97        98   146566.5            4  19007      1.540049    0.142517   
98        99   192783.7           56  19111      2.029479    0.148545   

    FIPS_INT CountyName State CountyDisplayName  \
0      19119       LYON    IA       Lyon County   
1      19143    OSCEO

In [6]:
# Ensure both columns are of string type
geojson_data['CountyName'] = geojson_data['CountyName'].astype(str)
geojson_data['State'] = geojson_data['State'].astype(str)
tornado_data['CountyName'] = tornado_data['CountyName'].astype(str)
tornado_data['State'] = tornado_data['State'].astype(str)

# Check unique values and their lengths before merging
unique_tornado_counties = tornado_data['CountyName'].unique()
unique_geojson_counties = geojson_data['CountyName'].unique()

# Create a DataFrame to display lengths
county_check = pd.DataFrame({
    'Tornado County Name': unique_tornado_counties,
    'Tornado Length': [len(name) for name in unique_tornado_counties],
    'GeoJSON County Name': unique_geojson_counties,
    'GeoJSON Length': [len(name) for name in unique_geojson_counties]
})

print(county_check)
# st.write("County Name Length Check:", county_check)

   Tornado County Name  Tornado Length GeoJSON County Name  GeoJSON Length
0                 LYON               4                LYON               4
1              OSCEOLA               7             OSCEOLA               7
2            DICKINSON               9           DICKINSON               9
3                EMMET               5               EMMET               5
4            ALLAMAKEE               9           ALLAMAKEE               9
..                 ...             ...                 ...             ...
94               DAVIS               5               DAVIS               5
95             DECATUR               7             DECATUR               7
96               WAYNE               5               WAYNE               5
97           APPANOOSE               9           APPANOOSE               9
98                 LEE               3                 LEE               3

[99 rows x 4 columns]


In [7]:
# Merging GeoJSON data with tornado risk data
# @st.cache_data
def merge_data(_geojson, _tornado_data):
    merged_data = _geojson.merge(_tornado_data, on=['CountyName', 'State'], how='left')
    return merged_data

merged_gdf = merge_data(geojson_data, tornado_data)
print(merged_gdf)
# st.write('Merged Data', merged_gdf[['DateTime', 'TornadoRisk', 'CountyName']].head(20))

       OBJECTID  PERIMETER  DOMCountyID   FIPS  Shape_Length  Shape_Area  \
0             1   193975.5           60  19119      2.212753    0.169102   
1             1   193975.5           60  19119      2.212753    0.169102   
2             1   193975.5           60  19119      2.212753    0.169102   
3             1   193975.5           60  19119      2.212753    0.169102   
4             1   193975.5           60  19119      2.212753    0.169102   
...         ...        ...          ...    ...           ...         ...   
83155        99   192783.7           56  19111      2.029479    0.148545   
83156        99   192783.7           56  19111      2.029479    0.148545   
83157        99   192783.7           56  19111      2.029479    0.148545   
83158        99   192783.7           56  19111      2.029479    0.148545   
83159        99   192783.7           56  19111      2.029479    0.148545   

       FIPS_INT CountyName State CountyDisplayName_x  \
0         19119       LYON    I

In [8]:
# # Check for any non-matching entries
non_matching = merged_gdf[merged_gdf['TornadoRisk'].isna()]
print(non_matching)
# st.write('Non-matching entries after merge:', non_matching.drop(columns='geometry').head(20))

Empty GeoDataFrame
Columns: [OBJECTID, PERIMETER, DOMCountyID, FIPS, Shape_Length, Shape_Area, FIPS_INT, CountyName, State, CountyDisplayName_x, geometry, Date, Time, DateTime, TornadoRisk, CountyDisplayName_y]
Index: []


In [11]:
# Static variable for date and time
selected_date = st.sidebar.date_input("Select Date", value=pd.Timestamp('2023-02-26'))
selected_time = st.sidebar.time_input("Select Time", value=pd.Timestamp('07:11:58').time())
selected_datetime = pd.Timestamp.combine(selected_date, selected_time)
print(selected_datetime)

# Filter data based on selected datetime
filtered_data = merged_gdf[merged_gdf['DateTime'] == selected_datetime]
print(filtered_data)
# st.write('Filtered Data: ', filtered_data.drop(columns='geometry').head())

2023-02-26 07:11:58
       OBJECTID  PERIMETER  DOMCountyID   FIPS  Shape_Length  Shape_Area  \
7             1   193975.5           60  19119      2.212753    0.169102   
847           2   130929.6           72  19143      1.434543    0.114867   
1687          3   131290.5           30  19059      1.438023    0.116176   
2527          4   130956.8           32  19063      1.434046    0.115803   
3367          5   172890.3            3  19005      1.847862    0.189376   
...         ...        ...          ...    ...           ...         ...   
78967        95   144995.3           26  19051      1.526090    0.139393   
79807        96   148600.8           27  19053      1.558094    0.147367   
80647        97   147785.1           93  19185      1.551256    0.145582   
81487        98   146566.5            4  19007      1.540049    0.142517   
82327        99   192783.7           56  19111      2.029479    0.148545   

       FIPS_INT CountyName State CountyDisplayName_x  \
7         1

In [12]:
# Define color scale logic and apply it to the filtered DataFrame
def apply_color_scale(df):
    colors = []
    for risk in df['TornadoRisk']:
        if pd.isna(risk):
            colors.append('rgba(0,0,0,0)')  # Transparent if no data
        elif risk < 0.5:
            colors.append('blue')  # Blue
        elif risk < 0.75:
            colors.append('yellow')  # Yellow
        else:
            colors.append('red')  # Red
    df['color'] = colors

apply_color_scale(filtered_data)
print(filtered_data)
# st.write('Filtered Data with Color:', filtered_data.drop(columns='geometry').head())

       OBJECTID  PERIMETER  DOMCountyID   FIPS  Shape_Length  Shape_Area  \
7             1   193975.5           60  19119      2.212753    0.169102   
847           2   130929.6           72  19143      1.434543    0.114867   
1687          3   131290.5           30  19059      1.438023    0.116176   
2527          4   130956.8           32  19063      1.434046    0.115803   
3367          5   172890.3            3  19005      1.847862    0.189376   
...         ...        ...          ...    ...           ...         ...   
78967        95   144995.3           26  19051      1.526090    0.139393   
79807        96   148600.8           27  19053      1.558094    0.147367   
80647        97   147785.1           93  19185      1.551256    0.145582   
81487        98   146566.5            4  19007      1.540049    0.142517   
82327        99   192783.7           56  19111      2.029479    0.148545   

       FIPS_INT CountyName State CountyDisplayName_x  \
7         19119       LYON    I

/opt/anaconda3/lib/python3.11/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [23]:
# Ensure the filtered data contains the geometry and convert DateTime to string
filtered_geojson = gpd.GeoDataFrame(filtered_data, geometry='geometry')
filtered_geojson['DateTime'] = filtered_geojson['DateTime'].astype(str)
print(filtered_geojson.head())
filtered_geojson.dtypes

      OBJECTID  PERIMETER  DOMCountyID   FIPS  Shape_Length  Shape_Area  \
7            1   193975.5           60  19119      2.212753    0.169102   
847          2   130929.6           72  19143      1.434543    0.114867   
1687         3   131290.5           30  19059      1.438023    0.116176   
2527         4   130956.8           32  19063      1.434046    0.115803   
3367         5   172890.3            3  19005      1.847862    0.189376   

      FIPS_INT CountyName State CountyDisplayName_x  \
7        19119       LYON    IA         Lyon County   
847      19143    OSCEOLA    IA      Osceola County   
1687     19059  DICKINSON    IA    Dickinson County   
2527     19063      EMMET    IA        Emmet County   
3367     19005  ALLAMAKEE    IA    Allamakee County   

                                               geometry        Date      Time  \
7     POLYGON ((-10753368.681 5388440.461, -10751956...  2023-02-26  07:11:58   
847   POLYGON ((-10671185.490 5388386.000, -10668976... 

OBJECTID                  int64
PERIMETER               float64
DOMCountyID               int64
FIPS                     object
Shape_Length            float64
Shape_Area              float64
FIPS_INT                  int64
CountyName               object
State                    object
CountyDisplayName_x      object
geometry               geometry
Date                     object
Time                     object
DateTime                 object
TornadoRisk             float64
CountyDisplayName_y      object
color                    object
dtype: object

In [21]:
# Convert GeoDataFrame to GeoJSON
filtered_geojson_json = json.loads(filtered_geojson.to_json())
# print(filtered_geojson_json)
# st.write('filtered_geojson_json:', json.dumps(filtered_geojson_json, indent=2))

In [24]:
import folium
import geopandas as gpd

# Ensure that your 'filtered_geojson' dataframe is a GeoDataFrame with geometry and color columns

# Center the map on Iowa
m = folium.Map(location=[41.878, -93.097], zoom_start=7)

# Function to style the counties
def style_function(feature):
    return {
        'fillColor': feature['properties']['color'],
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    }

# Add the GeoJSON to the map
folium.GeoJson(
    filtered_geojson,
    name='Iowa Tornado Risk',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['CountyName', 'TornadoRisk'], aliases=['County', 'Tornado Risk'])
).add_to(m)

# Add a layer control panel
folium.LayerControl().add_to(m)

# Display the map
m.save('iowa_tornado_risk_map.html')
m


In [ ]:
# Create a map with Plotly Express
fig = px.choropleth(
    filtered_geojson,
    geojson=filtered_geojson_json,
    locations='CountyName',
    featureidkey="properties.CountyName",
    color='TornadoRisk',
    color_continuous_scale=[
        [0, 'rgba(0,0,0,0)'],  # Transparent
        [0.5, 'blue'],
        [0.75, 'yellow'],
        [1, 'red']
    ],
    hover_name='CountyName',
    hover_data={'CountyName': False, 'TornadoRisk': True},
    labels={'TornadoRisk': 'Tornado Risk'}
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, coloraxis_colorbar=dict(
    title="Tornado Risk",
    tickvals=[0, 0.5, 0.75, 1],
    ticktext=['No Data', '<0.5', '0.5-0.75', '>0.75']
))

fig.show()

# st.plotly_chart(fig)

# st.write("Select a date and time to view the tornado risk in Iowa. The map shows the tornado risk for each county in Iowa. Use the sidebar to select a specific date and time. The color scale represents the tornado risk level: Blue (<0.5), Yellow (<0.75), Red (<1.0).")